In [1]:
import os

from DeepPurpose.dataset import *
import DTI_inspire as models
import matplotlib.pyplot as plt
import pandas as pd
from utils_inspire import *

In [2]:
X_drug, X_target, y  = load_process_DAVIS(path = './data', binary = False, convert_to_log = True, threshold = 30)

if not os.path.isfile('./data/BindingDB_All.tsv'):
    bdb_path = download_BindingDB(path = './data')
else:
    bdb_path = './data/BindingDB_All.tsv'
X_drug2, X_target2, y2  = process_BindingDB(path = bdb_path, df = None, y = 'Kd', binary = False, convert_to_log = True, threshold = 30)

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!
Beginning to download dataset...
Beginning to extract zip file...
Done!
Loading Dataset from path...


b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193 fields, saw 241\nSkipping line 1378088: expected 193 fields, saw 241\nSkipping line 1378089: expected 193 fields, saw 241\nSkipping line 1378090: e

Beginning Processing...
There are 66444 drug target pairs.
Default set to logspace (nM -> p) for easier regression


In [3]:
# X_drug

In [4]:
# X_drug2

In [5]:
# X_target

In [6]:
# X_target2

In [7]:
# y

In [8]:
# y2

In [9]:
X_drug = np.concatenate((X_drug, X_drug2), axis=0)
X_target = np.concatenate((X_target, X_target2), axis=0)
y = np.concatenate((y, y2), axis=0)

In [10]:
drug_encoding, target_encoding = 'Transformer', 'CNN_inspire'
#ensure none of test smiles appears in set
train, val, test  = data_process(X_drug, X_target, y, 
                      drug_encoding, target_encoding, 
                      split_method='cold_drug',frac=[0.8,0.1,0.1],
                      random_seed = 42)
# train.head()
print(f"First SMILE representation: \n{train.drug_encoding.iloc[0]}")
print(f"First target representation: \n{train.target_encoding.iloc[0]}")

Drug Target Interaction Prediction Mode...
in total: 96500 drug-target pairs
encoding drug...
unique drugs: 10733
encoding protein...
unique target sequence: 1449
splitting dataset...
Done.
First SMILE representation: 
(array([1124, 1823,  229, 1925,  606,   38,  282,  248,   85,  258,  549,
       1860,    6,  248,  248,   85,  333,  621, 1578,  650,  222, 1456,
         52,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]))
First target representation: 
[11 18 18 ...  0  0  0]


In [14]:
train.to_pickle('data/G35_data_train.pkl')
val.to_pickle('data/G35_data_vali.pkl')
test.to_pickle('data/G35_data_test.pkl')

In [11]:
from inspire_encoder import InspireEncoder
config = {'inspire_activation': 'relu',
            'CNN_inspire_filters': 128,
            'protein_strides': [10, 15, 20, 25, 30],
            'inspire_dropout': 0.2,
            'protein_layers': [128] }
model = InspireEncoder(**config)
if torch.cuda.is_available():
    model.cuda()


model(torch.from_numpy(np.vstack(train.target_encoding.values[0:2])))

/home/william/ML/DTI/inspire_encoder.py:43: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.emb.weight)
/home/william/ML/DTI/inspire_encoder.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(network[0].weight)
/home/william/ML/DTI/inspire_encoder.py:83: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(network[0].weight)


tensor([[0.0000, 0.0000, 0.0000, 1.2491, 1.2353, 0.0000, 1.0360, 1.2484, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1016, 0.0000, 0.0000,
         0.0000, 1.2245, 0.0000, 0.0000, 0.0000, 1.2481, 1.2483, 0.0000, 0.0000,
         1.2437, 0.0000, 0.0000, 1.2441, 0.0000, 0.0000, 0.0000, 0.0000, 1.1707,
         0.0000, 0.0000, 0.0000, 0.5771, 0.0000, 1.2461, 1.2488, 0.0000, 0.0000,
         1.2463, 1.2486, 1.2452, 0.0000, 0.0000, 0.0000, 0.0000, 1.2427, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.2494, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.2476, 1.2484, 1.2293, 1.2451,
         0.0000, 0.0000, 0.0000, 0.0000, 1.2426, 0.0000, 1.2285, 0.0000, 1.2467,
         0.0000, 0.0000, 1.2424, 0.0000, 0.0000, 1.2397, 0.0000, 0.0000, 1.2486,
         1.2428, 0.0000, 0.0000, 1.1842, 0.0000, 1.2463, 1.2480, 1.2422, 1.2473,
         1.2457, 1.2473, 0.0000, 0.0000, 1.2430, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [5]:
train = pd.read_pickle('data/G35_data_train.pkl')
val = pd.read_pickle('data/G35_data_vali.pkl')
test = pd.read_pickle('data/G35_data_test.pkl')

drug_encoding, target_encoding = 'Transformer', 'CNN_inspire'
config = generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [256],  
                         train_epoch = 100, 
                         LR = 0.0001, 
                         batch_size = 32,
                         inspire_activation = 'elu',
                         CNN_inspire_filters = 128,
                         protein_strides = [10, 15, 20, 25, 30],
                         inspire_dropout =  0,
                         protein_layers =  [128],
                         transformer_emb_size_drug = 128,
                         transformer_intermediate_size_drug = 512,
                         transformer_num_attention_heads_drug = 8,
                         transformer_n_layer_drug = 8,
                         transformer_dropout_rate = 0.1,
                         transformer_attention_probs_dropout = 0.1,
                         transformer_hidden_dropout_rate = 0.1,
                         num_workers = 0,
                         result_folder = 'result/first_trial_run'
                         )
model = models.model_initialize(**config)
# print(model)

In [6]:
model.train(train, val, test, save_path='models/1_layer_out', drop_last = True)

Let's use 1 GPU!
--- Data Preparation ---
--- Go for Training ---
Training at Epoch 1 iteration 0 with loss 30.3524. Total time 0.00027 hours
Training at Epoch 1 iteration 100 with loss 1.95095. Total time 0.00361 hours
Training at Epoch 1 iteration 200 with loss 1.75270. Total time 0.00722 hours
Training at Epoch 1 iteration 300 with loss 2.08991. Total time 0.01083 hours
Training at Epoch 1 iteration 400 with loss 1.03628. Total time 0.01444 hours
Training at Epoch 1 iteration 500 with loss 2.24052. Total time 0.01805 hours
Training at Epoch 1 iteration 600 with loss 1.02983. Total time 0.02166 hours
Training at Epoch 1 iteration 700 with loss 1.15521. Total time 0.02527 hours
Training at Epoch 1 iteration 800 with loss 1.66349. Total time 0.02888 hours
Training at Epoch 1 iteration 900 with loss 1.97682. Total time 0.0325 hours
Training at Epoch 1 iteration 1000 with loss 1.60092. Total time 0.03611 hours
Training at Epoch 1 iteration 1100 with loss 1.19028. Total time 0.03944 hours

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])